# ТИПЫ ДАННЫХ И ИХ ПРЕОБРАЗОВАНИЕ

Далее в заданиях используются данные с платформы Airbnb в Берлине ✈️.

Имеются следующие таблицы:
    
1. `listings` – информация о жилье, включая полные описания, характеристики и средние оценки в отзывах; поскольку столбцов очень много, нужные перечислены в текстах самих задач;  
2. `calendar_summary` – информация о доступности и цене того или иного жилья по дням  
    listing_id – идентификатор жилья (объявления)  
    date       – дата  
    available  – доступность жилья в данный день (t/f)  
    price      – цена за ночь  

3. `reviews` – отзывы  
    listing_id   –  идентификатор жилья      
    id  – id отзыва   
    date  – дата отзыва      
    reviewer_id  – id ревьюера (автора отзыва)  
    reviewer_name  – имя автора  
    comments – сам отзыв  

# 1

На Airbnb есть две основные группы пользователей:

· Хозяева (хосты, hosts) – сдают жилье  
· Путешественники (travelers) – снимают  
Начнем с анализа характеристик хозяев в таблице listings!   

Пользователи, сдающие квартиры на Airbnb, зарегистрировались в разное время. Кто-то – очень давно, а кто-то совсем недавно. Давайте проверим, в какой месяц и год зарегистрировалось наибольшее количество новых хостов. В качестве ответа введите дату следующего формата: 2010-12

· host_id – идентификатор хозяина  
· host_since – дата регистрации как хост  
Note: Сам хост может встретиться в таблице несколько раз, если сдает несколько помещений, поэтому не забудьте оставить уникальные значения host_id. Также обратите внимание на тип данных в host_since, возможно вам пригодится toStartOfMonth() для извлечения части даты и toDateOrNull() для обработки пустых значений.

In [ ]:
select  toStartOfMonth(toDateOrNull(host_since)) as reg_host, count(distinct host_id) as host
from listings
group by toStartOfMonth(toDateOrNull(host_since))
order by count(distinct host_id) desc
limit 100

# 2

Посмотрим на среднюю частоту ответа среди хозяев (f) и суперхозяев (t).

Значения частоты ответа хранятся как строки и включают значок %, который необходимо заменить на пустоту (''). После этого приведите столбец к нужному типу данных с помощью toInt32OrNull() и посчитайте среднюю частоту отклика в разбивке по тому, является ли хост суперхозяином или нет. В качестве ответа укажите наибольшее среднее.

· host_response_rate – частота ответа  
· host_is_superhost – является ли суперхозяином  
Важный момент: у каждого host_id есть только одно уникальное значение частоты ответа, а также одна единая отметка суперхозяина. Чтобы посчитать показатели честно, нужно использовать подзапрос и взять уникальные комбинации с помощью DISTINCT.

In [ ]:
select host_is_superhost ,avg(toInt32OrNull(replaceAll(host_response_rate, '%', ''))) as Avg_response_rate 
from (
    select distinct host_id, host_is_superhost, host_response_rate
    from listings
)
where host_is_superhost = 't' or host_is_superhost = 'f'
group by host_is_superhost
limit 50

# 3

Сгруппируйте данные из listings по хозяевам (host_id) и посчитайте, какую цену за ночь в среднем каждый из них устанавливает (у одного хоста может быть несколько объявлений). Идентификаторы сдаваемого жилья объедините в отдельный массив. Таблицу отсортируйте по убыванию средней цены и убыванию host_id (в таком порядке). В качестве ответа укажите первый массив в результирующей таблице, состоящий более чем из двух id.  
· host_id – идентификатор хозяина  
· id – идентификатор жилья  
· price – цена за ночь в конкретном месте

In [ ]:
select host_id, groupArray(id) as all_id ,avg(toFloat64OrNull(replaceAll(price,'$',''))) as avg_price
from listings
group by host_id
order by avg(toFloat64OrNull(replaceAll(price,'$',''))) desc, host_id desc
limit 50

# 4

Немного усложним предыдущую задачу, и посчитаем разницу между максимальной и минимальной установленной ценой у каждого хозяина. В качестве ответа укажите идентификатор хоста, у которого разница оказалась наибольшей. 

· host_id – идентификатор хозяина  
· id – идентификатор жилья  
· price – цена за ночь в конкретном месте

In [ ]:
select host_id, groupArray(id) as all_id ,(max(toFloat64OrNull(replaceAll(price,'$',''))) - min(toFloat64OrNull(replaceAll(price,'$','')))) as diff_price
from listings
group by host_id
order by max(toFloat64OrNull(replaceAll(price,'$',''))) - min(toFloat64OrNull(replaceAll(price,'$',''))) desc, host_id desc
limit 50

# 5

Теперь сгруппируйте данные по типу жилья и выведите средние значения цены за ночь, размера депозита и цены уборки. Обратите внимание на тип данных, наличие значка $ и запятых в больших суммах. Для какого типа жилья среднее значение залога наибольшее? 

· room_type – тип сдаваемого жилья   
· price – цена за ночь   
· security_deposit – залог за сохранность имущества  
· cleaning_fee – плата за уборку

In [ ]:
select room_type, avg(toFloat64OrNull(replaceAll(price,'$',''))) as avg_price,
avg(toFloat64OrNull(replaceAll(security_deposit,'$',''))) as avg_dep,
avg(toFloat64OrNull(replaceAll(cleaning_fee,'$',''))) as avg_cleaning
from listings
group by room_type
order by avg_price desc

# 6

Отлично! В каких частях города средняя стоимость за ночь является наиболее низкой? 

Сгруппируйте данные по neighbourhood_cleansed и посчитайте среднюю цену за ночь в каждом районе. В качестве ответа введите название места, где средняя стоимость за ночь ниже всего.

· price – цена за ночь  
· neighbourhood_cleansed – район/округ города

In [ ]:
select neighbourhood_cleansed, min(toFloat64OrNull(replaceAll(price,'$',''))) as min_price
from listings
group by neighbourhood_cleansed
having min_price > 0
order by min_price asc

# 7

В каких районах Берлина средняя площадь жилья, которое сдаётся целиком, является наибольшей? Отсортируйте по среднему и выберите топ-3. 

· neighbourhood_cleansed – район   
· square_feet – площадь  
· room_type – тип (нужный – 'Entire home/apt')

In [ ]:
select neighbourhood_cleansed, avg(toFloat64OrNull(square_feet)) as avg_square
from listings
where room_type = 'Entire home/apt'
group by neighbourhood_cleansed
order by avg_square desc
limit 3